In [1]:
pip install torch torchvision ultralytics opencv-python numpy


In [2]:
import cv2
import torch
import numpy as np
from ultralytics import YOLO
from torchvision import models, transforms

# Load YOLOv5 Model
yolo_model = YOLO("yolov5s.pt")  # Automatically downloads the model

# Load Faster R-CNN Model
faster_rcnn_model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
faster_rcnn_model.eval()

def resize_image(image, width=800):
    """Resize image to fixed width while keeping aspect ratio"""
    h, w = image.shape[:2]
    aspect_ratio = width / float(w)
    new_height = int(h * aspect_ratio)
    return cv2.resize(image, (width, new_height))

def run_yolo(image_path):
    """Run YOLOv5 detection and display resized output"""
    img = cv2.imread(image_path)
    results = yolo_model(image_path)

    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            score = box.conf[0].item()
            if score > 0.5:
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

    img_resized = resize_image(img)
    cv2.imshow("YOLOv5 Detection", img_resized)
    cv2.imwrite("yolo_output.jpg", img_resized)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def run_faster_rcnn(image_path):
    """Run Faster R-CNN detection and display resized output"""
    img = cv2.imread(image_path)
    img_copy = img.copy()  # Keep original for drawing
    transform = transforms.Compose([transforms.ToPILImage(), transforms.ToTensor()])
    img_tensor = transform(img).unsqueeze(0)

    with torch.no_grad():
        detections = faster_rcnn_model(img_tensor)

    for i in range(len(detections[0]["scores"])):
        score = detections[0]["scores"][i].item()
        if score > 0.5:
            box = detections[0]["boxes"][i].tolist()
            x1, y1, x2, y2 = map(int, box)
            cv2.rectangle(img_copy, (x1, y1), (x2, y2), (255, 0, 0), 2)

    img_resized = resize_image(img_copy)
    cv2.imshow("Faster R-CNN Detection", img_resized)
    cv2.imwrite("faster_rcnn_output.jpg", img_resized)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Run both detections
image_path = "C:\\Users\\hp\\Desktop\\121ME0017\\detection image 1.jpg"
run_yolo(image_path)
run_faster_rcnn(image_path)



PRO TIP  Replace 'model=yolov5s.pt' with new 'model=yolov5su.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.



D:\anan\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
D:\anan\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



image 1/1 C:\Users\hp\Desktop\121ME0017\detection image 1.jpg: 480x640 15 persons, 1 car, 3 trucks, 1 traffic light, 2 backpacks, 308.1ms
Speed: 28.0ms preprocess, 308.1ms inference, 68.9ms postprocess per image at shape (1, 3, 480, 640)
